<a href="https://colab.research.google.com/github/Mehali25/MainProj/blob/main/sensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyserial opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 4.3 MB/s eta 0:00:00


In [2]:
import serial
import time
import cv2
import numpy as np

# Initialize serial communication with the USB sensor
# Replace 'COM3' with your sensor's actual port or `/dev/ttyUSB0` on Linux
sensor = serial.Serial(port='COM3', baudrate=9600, timeout=1)

def capture_fingerprint():
    print("Waiting for fingerprint capture...")
    # Send a command to start capturing the fingerprint (depends on sensor)
    # For most sensors, sending a command will initiate the fingerprint scan
    sensor.write(b'\x01')  # Example command; replace with the correct one for your sensor
    time.sleep(2)

    # Read fingerprint data
    data = sensor.read(512)  # Adjust size based on the sensor's data output
    if data:
        print("Fingerprint data captured.")
        # Process or save the image data as needed
        # Example: converting raw data into an image format using OpenCV
        image = np.frombuffer(data, dtype=np.uint8).reshape((240, 216))  # Adjust dimensions
        cv2.imshow("Fingerprint", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("No fingerprint data found.")

# Capture a fingerprint
capture_fingerprint()


SerialException: [Errno 2] could not open port COM3: [Errno 2] No such file or directory: 'COM3'

In [3]:
def extract_features(image):
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(image, (5, 5), 0)

    # Convert to binary image
    _, binary = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY_INV)

    # Detect edges using Canny
    edges = cv2.Canny(binary, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    features = [cv2.boundingRect(cnt) for cnt in contours]  # Example: bounding boxes

    print("Extracted features:", features)
    return features

# Run feature extraction on the captured fingerprint
extract_features(image)


NameError: name 'image' is not defined

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import numpy as np

# Example data: replace with real fingerprint features and blood groups
data = np.array([[0.1, 0.5], [0.7, 0.2], [0.4, 0.8]])  # Features
labels = np.array(['A', 'B', 'O'])  # Blood groups

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

# Train a K-Nearest Neighbors classifier
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)

# Example prediction
sample_feature = [0.5, 0.5]  # Replace with extracted feature data
prediction = model.predict([sample_feature])
print("Predicted Blood Group:", prediction[0])


ValueError: Expected n_neighbors <= n_samples_fit, but n_neighbors = 3, n_samples_fit = 2, n_samples = 1

In [5]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Blood Group Detection</title>
</head>
<body style="text-align:center; padding-top:20%;">
    <h1>Blood Group Detection Simulation</h1>
    <button onclick="startBiometricScan()">Simulate Biometric Scan</button>
    <p id="result"></p>

    <script>
        function startBiometricScan() {
            fetch("/predict_blood_group")
                .then(response => response.json())
                .then(data => {
                    document.getElementById("result").innerText = "Predicted Blood Group: " + data.blood_group;
                });
        }
    </script>
</body>
</html>


SyntaxError: invalid syntax (<ipython-input-5-c67b53ad3ceb>, line 1)

In [ ]:
from flask import Flask, jsonify
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

app = Flask(__name__)

# Dummy classifier (replace with your trained model)
data = np.array([[0.1, 0.5], [0.7, 0.2], [0.4, 0.8]])
labels = np.array(['A', 'B', 'O'])
model = KNeighborsClassifier(n_neighbors=3).fit(data, labels)

@app.route('/predict_blood_group', methods=['GET'])
def predict_blood_group():
    # Example feature extraction (replace with real sensor data)
    sample_feature = [0.5, 0.5]
    prediction = model.predict([sample_feature])

    return jsonify(blood_group=prediction[0])

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
